In [ ]:
# Importar as bibliotecas necessárias
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import norm
pd.set_option('display.max_columns', None)

In [ ]:
# Abrir as tabelas .csv como dataframes de trabalho
users = pd.read_csv('../CSVs/tbl_users.csv', sep=';', low_memory=False)
events = pd.read_csv('../CSVs/tbl_events.csv', low_memory=False)
users_events = pd.read_csv('../CSVs/tbl_users_events.csv', low_memory=False)
analise = pd.read_csv('../CSVs/gg_analise_db1.csv', sep=';', low_memory=False)
users_games = pd.read_csv('../CSVs/tbl_users_games.csv', low_memory=False)
games = pd.read_json('../CSVs/tbl_games.json')
friends = pd.read_csv('../CSVs/tbl_friends.csv', low_memory=False)
chats = pd.read_csv('../CSVs/tbl_chat_members.csv', low_memory=False)
plataformas = pd.read_csv('../CSVs/tbl_platforms.csv', low_memory=False)
platforms = pd.read_csv('../CSVs/tbl_users_platforms.csv', low_memory=False)
logs = pd.read_csv('../CSVs/tbl_users_login_log.csv', low_memory=False)

In [ ]:
# Ao tentar fazer a mudança para INT, apareceram erros devido a alguns dados da coluna não serem numéricos.
# Aí eu busquei esses, que eram poucos, para eliminar as linhas.
users.loc[users['id']=='   "']
users.loc[users['id']=='?>"']

# Eliminando as linhas problemáticas.
users.drop(index=16790, inplace=True)
users.drop(index=22557, inplace=True)

# Aí, sim, alterando a coluna id para INT.
users.id = users.id.astype(int)

In [ ]:
gg_users = users.set_index('id', drop=True)
gg_users.rename_axis('user_id', inplace=True)

In [ ]:
# Vamos agrupar diversas informações por id de usuário para facilitar as análises.

grouped_games = users_games.groupby(by='user_id').count()
grouped_games.rename(columns={'id':'qtde_games'}, inplace=True)
grouped_games.drop(columns=['game_id', 
                            'platform_id', 
                            'network_id', 
                            'is_favorite', 
                            'status_id'], 
                   inplace=True
                  )

events.rename(columns={'created_by':'user_id'}, inplace=True)
grouped_events = events.groupby(by='user_id').count()
grouped_events.rename(columns={'id':'qtde_grupos'}, inplace=True)
grouped_events.drop(grouped_events.iloc[:, 1:], inplace = True, axis = 1) 

grouped_friends = friends.groupby(by='user_id').count()
grouped_friends.drop(columns=['id'], inplace=True)
grouped_friends.rename(columns={'friend_id':'qtde_contatos'}, inplace=True)

grouped_user_events = users_events.groupby(by='user_id').count()
grouped_user_events.rename(columns={'id':'qtde_eventos'}, inplace=True)
grouped_user_events.drop(grouped_user_events.iloc[:, 1:], inplace = True, axis = 1) 

chats_grouped = chats.groupby(by='user_id').count()
chats_grouped.rename(columns={'chat_id':'qtde_mensagens'}, inplace=True)
chats_grouped.drop(chats_grouped.iloc[:, 1:], inplace = True, axis = 1) 

platforms_grouped = platforms.groupby(by='user_id').count()
platforms_grouped.rename(columns={'id':'qtde_plataformas'}, inplace=True)
platforms_grouped.drop(platforms_grouped.iloc[:, 1:], inplace = True, axis = 1) 

grouped_logs = logs.groupby(by='user_id').count()
grouped_logs.rename(columns={'id':'qtde_acessos'}, inplace=True)
grouped_logs.drop(columns=['login_datetime'], inplace=True)

In [ ]:
# Agora reunimos todos os dataframes em um único, com o user_id como chave.

users_gg = pd.concat([gg_users, 
                      grouped_games, 
                      grouped_events, 
                      grouped_friends, 
                      grouped_user_events, 
                      chats_grouped, 
                      platforms_grouped, 
                      grouped_logs], 
                      axis = 1
                    )

In [ ]:
# Vamos preencher os dados faltantes com 0.

users_gg["qtde_games"].fillna(0, inplace=True)
users_gg["qtde_grupos"].fillna(0, inplace=True)
users_gg["qtde_contatos"].fillna(0, inplace=True)
users_gg["qtde_eventos"].fillna(0, inplace=True)
users_gg["qtde_mensagens"].fillna(0, inplace=True)
users_gg["qtde_plataformas"].fillna(0, inplace=True)
users_gg["qtde_acessos"].fillna(0, inplace=True)

In [ ]:
users_gg

In [ ]:
users_2020 = users_gg.loc[users_gg['creation_datetime'] > '2020-07-01']

In [ ]:
users_2020.loc[users_2020['phone_number'].notnull()]

In [ ]:
users_2020.info()

### Verificando quais são os jogos com mais usuários no app.

In [ ]:
games

In [ ]:
games.loc[games['rating'].notnull()]

In [ ]:
games.rename(columns={'id':'game_id'}, inplace=True)
games.set_index('game_id', inplace=True)

# Vamos agrupar os dados de número de usuários por game_id. Assim saberemos quantos usuários cada jogo tem.

usersgames_grouped = users_games.groupby(by='game_id').count()
usersgames_grouped.rename(columns={'id':'qtde_users_by_game'}, inplace=True)
usersgames_grouped.drop(usersgames_grouped.iloc[:, 1:], inplace = True, axis = 1) 

# Juntamos os dois dataframes para formar a base para os gráficos, com os nomes dos jogos.
games_gg = pd.concat([games, usersgames_grouped], axis = 1)
games_gg.drop(columns=['original_id', 
                       'slug', 
                       'cover_base_url', 
                       'cover_image_png'], 
              inplace=True
             )
games_gg["qtde_users_by_game"].fillna(0, inplace=True)

In [ ]:
games_gg

In [ ]:
# Vamos ordenar a lista e selecionar os 30 maiores jogos para formar o gráfico.
games_plot = games_gg.sort_values("qtde_users_by_game", ascending=False).head(30)

# Vamos ordenar novamente a lista e selecionar os 100 maiores jogos para calcular a média de usuários por jogo.
games_plot2 = games_gg.sort_values("qtde_users_by_game", ascending=False).head(100)

In [ ]:
games_plot2.describe()

In [ ]:
# Gerando o gráfico para analisar os usuários por jogo.

ax = games_plot.plot(x='name', 
                     y='qtde_users_by_game', 
                     kind='barh', 
                     figsize=(12, 15), 
                     width=0.9,
                     legend=False, 
                     xlim=(0, 13000), 
                     color='#28b4e7',
                     fontsize=14
                    )
ax.invert_yaxis()
ax.set_title('Jogos preferidos pelos Usuários', fontsize=24, loc='center', pad=20)
ax.set_xlabel('Quantidade de usuários por jogo', fontsize=14) 
ax.set_ylabel('Nome do jogo', fontsize=14) 
for p in ax.patches:
    ax.annotate(str(int(p.get_width())), 
                (p.get_x() + p.get_width(), 
                p.get_y()+1), 
                xytext=(5, 10), 
                textcoords='offset points', 
                va='baseline', 
                fontsize=14, 
                fontweight='bold', 
                color='#4b4b4b'
                )
plt.savefig(os.path.join('../Imagens/users_by_game.pdf'), 
            dpi=300, 
            format='pdf', 
            bbox_inches='tight') # use format='svg' or 'pdf' for vectorial pictures

### Verificando a quantidade de usuários por plataforma.

In [ ]:
plataformas

In [ ]:
platforms

In [ ]:
plataformas.rename(columns={'id':'platform_id'}, inplace=True)
plataformas.set_index('platform_id', inplace=True)

usersplats_grouped = platforms.groupby(by='platform_id').count()
usersplats_grouped.rename(columns={'id':'qtde_users_by_plat'}, inplace=True)
usersplats_grouped.drop(usersplats_grouped.iloc[:, 1:], inplace = True, axis = 1) 

plats_gg = pd.concat([plataformas, usersplats_grouped], axis = 1)
plats_gg.drop(columns=['igdb_id', 
                       'icon', 
                       'network_id', 
                       'is_active'], 
              inplace=True
             )

In [ ]:
plats_gg.describe()

In [ ]:
plats_plot = plats_gg.sort_values("qtde_users_by_plat", ascending=False)

In [ ]:
ax = plats_plot.plot(x='name', 
                     y='qtde_users_by_plat', 
                     kind='barh', 
                     figsize=(12, 15), 
                     width=0.9,
                     legend=False, 
                     xlim=(0, 40000), 
                     color='#28b4e7',
                     fontsize=14
                    )
ax.invert_yaxis()
ax.set_title('Plataformas preferidas pelos Usuários', fontsize=24, loc='center', pad=20)
ax.set_xlabel('Quantidade de usuários por plataforma', fontsize=14) 
ax.set_ylabel('Nome da Plataforma', fontsize=14) 
for p in ax.patches:
    ax.annotate(str(int(p.get_width())), 
                (p.get_x() + p.get_width(), 
                p.get_y()+1), 
                xytext=(5, 10), 
                textcoords='offset points', 
                va='baseline', 
                fontsize=14, 
                fontweight='bold', 
                color='#4b4b4b'
                )
plt.savefig(os.path.join('../Imagens/users_by_platform.pdf'), 
            dpi=300, 
            format='pdf', 
            bbox_inches='tight') # use format='svg' or 'pdf' for vectorial pictures

In [ ]:
print(f'Quantidade de Jogos máxima:       {users_gg["qtde_games"].max()}')
print(f'Quantidade de Grupos máxima:      {users_gg["qtde_grupos"].max()}')
print(f'Quantidade de Contatos máxima:    {users_gg["qtde_contatos"].max()}')
print(f'Quantidade de Eventos máxima:     {users_gg["qtde_eventos"].max()}')
print(f'Quantidade de Mensagens máxima:   {users_gg["qtde_mensagens"].max()}')
print(f'Quantidade de Plataformas máxima: {users_gg["qtde_plataformas"].max()}')
print(f'Quantidade de Acessos máxima:     {users_gg["qtde_acessos"].max()}')

In [ ]:
print(f'Quantidade de Jogos média:       {round(users_gg["qtde_games"].mean(), ndigits=2)}')
print(f'Quantidade de Grupos média:      {round(users_gg["qtde_grupos"].mean(), ndigits=2)}')
print(f'Quantidade de Contatos média:    {round(users_gg["qtde_contatos"].mean(), ndigits=2)}')
print(f'Quantidade de Eventos média:     {round(users_gg["qtde_eventos"].mean(), ndigits=2)}')
print(f'Quantidade de Mensagens média:   {round(users_gg["qtde_mensagens"].mean(), ndigits=2)}')
print(f'Quantidade de Plataformas média: {round(users_gg["qtde_plataformas"].mean(), ndigits=2)}')
print(f'Quantidade de Acessos média:     {round(users_gg["qtde_acessos"].mean(), ndigits=2)}')

In [ ]:
users_stats = users_gg[['qtde_games', 
                        'qtde_grupos', 
                        'qtde_contatos', 
                        'qtde_eventos', 
                        'qtde_mensagens', 
                        'qtde_plataformas', 
                        'qtde_acessos']]